## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, PowerTransformer, QuantileTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error

## Preparación de Datos para Regresión

In [5]:
# Leer el CSV
data = pd.read_csv('../../data/EstudioCrediticio_TrainP.csv')

# Eliminar la primera columna
data = data.iloc[:, 1:]
# Eliminar la penúltima columna
data = data.iloc[:, :-2].join(data.iloc[:, -1])

# Separar las características (X) y la variable a predecir (y)
X = data.iloc[:, :-1]  # Todas las columnas menos la última
y = data.iloc[:, -1]   # La última columna

# Identificar columnas numéricas y categóricas
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Linear Regression

### Creación de Pipelines

Se crearán las siguientes Pipelines para el preprocesado de los datos:
- En las variables categóricas, se aplicará un imputado basado en la moda y un OneHotEncoder para conseguir arrays numéricos
- En las variables numéricas, se aplicará un imputado basado en la media y se escalarán las variables con PowerTransformer, que aplica una transformación de potencia para hacer que los datos sean más gaussianos, reduciendo el impacto de los outliers. Se usará el método yeo-johnson al tener valores positivos y negativos.

In [8]:
# Pipeline para variables categóricas (Imputación + OneHotEncoder)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputar con el valor más frecuente
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Pipeline para variables numéricas (Imputación + Escalado)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputar con la media
    ('scaler', PowerTransformer(method='yeo-johnson')), # Escalar debido a outliers
])

# Combinación de transformaciones para columnas categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

En segundo lugar, nos preguntamos: ¿Cuál es el **mejor modelo de regresión Lineal**?

Se usará una Pipelina final que une el **preprocesamiento** de datos con el **regresor**, y posteriormente se realizará **validación cruzada** para comprobar que modelo de regresión lineal da mejor rendimiento. Se probará con _LinearRegression()_, _Ridge()_ que incluye penalización L2, _Lasso()_ que incluye regularización L1, y _ElasticNet()_ que utiliza ambas penalizaciones.

In [10]:
models = [LinearRegression(), Ridge(), Lasso(), ElasticNet()]

for regressor in models:
    # Crear pipeline final con preprocesamiento y modelo
    model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', regressor)
    ])

    # Definir la estrategia de validación cruzada con KFold
    kfold = KFold(n_splits=5, shuffle=True, random_state=73)

    # Usar cross_val_score para evaluar el modelo utilizando KFold
    results = cross_val_score(model, X, y, cv=kfold, scoring='r2')

    # Mostrar los resultados
    print(f'Accuracy promedio {model.named_steps['regressor']}: {results.mean()}')

Accuracy promedio LinearRegression(): 0.7663280613901761
Accuracy promedio Ridge(): 0.7663297722718247
Accuracy promedio Lasso(): 0.621180007511583
Accuracy promedio ElasticNet(): 0.620506526531895


Se observa que tanto la Regresión Lineal como Ridge ofrecen un rendimiento similar y funcionan mejor en nuestro conjunto de datos. No obstante, los resultados obtenidos no son satisfactorios.

### Evaluación del modelo mediante train, test y métricas

También podemos obtener métricas como la raíz del error cuadrático medio o el error absoluto medio además del R^2 score.

Para ello, dividimos el conjunto de datos en train y test, entranamos y evaluamos.

Usaremos en este caso la Regresión Lineal:

In [11]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
    ])

# Entrenar el modelo
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   PowerTransformer())]),
                                                  Index(['Edad', 'IngresoBrutoAnual', 'ScoreCrediticio', 'Experiencia',
       'CuantiaCredito', 'DuracionCredito', 'NumeroPersonasDependientes',
       'PagoMensualDeuda', 'TasaUtilizacionTarjetaCredito',
       'NumeroLineasCreditoAb...
       'TipoInteres', 'PagoMensualCredito', 'RatioTotalDeudaIngresos'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['SituacionLaboral', 'NivelEducativo', 'EstadoCivil', 'EstadoVivienda',
       'ObjetoCredito'],
      dtype='object'))])),
                ('regressor', LinearRegression())])

In [12]:
y_pred = model.predict(X_test)

# Calcular métricas para regresión
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R²: {r2}")

RMSE: 3.867989791705261
MAE: 3.20657275390625
R²: 0.7614386669915587


## Ensemble: Random Forest Regressor

Se desarrollarán las pipelines utilizando el mismo enfoque que en el modelo anterior. Sin embargo, a diferencia de este, el PowerTransformer() u otro escalador no son necesarios, ya que los árboles de decisión no se ven influenciados por la escala de las variables.

In [3]:
# Pipeline para variables categóricas (Imputación + OneHotEncoder)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputar con el valor más frecuente
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Pipeline para variables numéricas (Imputación + Escalado)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputar con la media
])

# Combinación de transformaciones para columnas categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Crear pipeline final con preprocesamiento y modelo
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=73))
])

Se realizará un Randomized Search para encontrar los **mejores hiperparámetros** para nuestro RandomForest.

**GridSearch VS Randomized Search**: 
La diferencia radice en el tiempo de ejecución. Ambos exploran exactamente el mismo espacio de parámetros. El resultado es bastante similar, mientras que el tiempo de ejecución de la búsqueda aleatoria es drásticamente menor. Además, ambos utilizan por defecto **validación cruzada** de 5 folds.

In [ ]:
# Definir el espacio de búsqueda para RandomizedSearchCV
param_dist = {
    'regressor__n_estimators': [100, 200, 300, 400, 500],
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    model,                                    # Pipeline
    param_distributions=param_dist,           # Hiperparámetros (espacio de búsqueda)
    verbose=1,                                # Mostrar progreso en la consola
    n_jobs=-1,                                # Usar todos los núcleos disponibles
    random_state=73                           # Reproducibilidad
)

random_search.fit(X_train, y_train)

# Mejores hiperparámetros
print("Mejores parámetros:", random_search.best_params_)

c:\Users\guigr\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 5 candidates, totalling 25 fits


Los resultados del Randomized Search fueron **n_estimators=250** y **max_depth=20**

Por ello, entrenaremos un modelo con dichos hiperparámetros y observaremos su rendimiento en test de la misma forma que en el modelo anterior.

In [15]:
# Crear pipeline final con preprocesamiento y modelo
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=73, n_estimators=250, max_depth=20))
])

# Ajustar el modelo
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  Index(['Edad', 'IngresoBrutoAnual', 'ScoreCrediticio', 'Experiencia',
       'CuantiaCredito', 'DuracionCredito', 'NumeroPersonasDependientes',
       'PagoMensualDeuda', 'TasaUtilizacionTarjetaCredito',
       'NumeroLineasCreditoAbiertas', 'NumeroSolicitudesCred...
       'TipoInteres', 'PagoMensualCredito', 'RatioTotalDeudaIngresos'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['SituacionLaboral', 'NivelEducativo', 'EstadoCivil', 'EstadoVivienda',
       'ObjetoCredito'],
      dtype='object'))])),
                ('regressor',
                 RandomForestRegressor(max_depth=20, n_estimators=250,
                                       random_state=73))])

In [16]:
# Evaluar el modelo
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"R² del conjunto de train: {train_score:.2f}")
print(f"R² del conjunto de test: {test_score:.2f}")

y_pred = model.predict(X_test)

# Calcular métricas para regresión
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE de test: {rmse}")
print(f"MAE de test: {mae}")

R² del conjunto de train: 0.98
R² del conjunto de test: 0.87
RMSE de test: 2.8868714263294346
MAE de test: 1.7398255139830143


## Ensemble: Boosting

Para comenzar, probaremos a simple vista qué modelo funciona mejor entre AdaBoostRegressor, GrandientBoostingRegressor y HistGradientBoostingRegressor.

In [61]:
# Pipeline para variables categóricas (Imputación + OneHotEncoder)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputar con el valor más frecuente
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

# Pipeline para variables numéricas (Imputación + Escalado)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputar con la media
    ('scaler', PowerTransformer(method='yeo-johnson')), # Escalar debido a outliers
])

# Combinación de transformaciones para columnas categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [62]:
models = [AdaBoostRegressor(), GradientBoostingRegressor(), HistGradientBoostingRegressor()]

for regressor in models:
    # Crear pipeline final con preprocesamiento y modelo
    model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', regressor)
    ])

    # Definir la estrategia de validación cruzada con KFold
    kfold = KFold(n_splits=5, shuffle=True, random_state=73)

    # Usar cross_val_score para evaluar el modelo utilizando KFold
    results = cross_val_score(model, X, y, cv=kfold, scoring='r2')

    # Mostrar los resultados
    print(f'Accuracy promedio {model.named_steps['regressor']}: {results.mean()}')

Accuracy promedio AdaBoostRegressor(): 0.6147926839405948
Accuracy promedio GradientBoostingRegressor(): 0.8662128831201631
Accuracy promedio HistGradientBoostingRegressor(): 0.8963140241289576


Se puede observar que el _HistGradientBoostingRegressor()_ obtiene el mejor rendimiento. Además, este estimador tiene **soporte nativo para valores faltantes** (NaN), por lo que no sería necesario aplicar los imputadores en las Pipelines, y es mucho **más rápido** en conjuntos grandes de datos como el nuestro.

Veamos si el uso de los imputadores empeoraba o no el resultado:

In [64]:
# Pipeline para variables categóricas
categorical_transformer_no_imputer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Pipeline para variables numéricas
numeric_transformer_no_imputer = Pipeline(steps=[
    ('scaler', PowerTransformer(method='yeo-johnson')),
])

# Combinación de transformaciones para columnas categóricas y numéricas
preprocessor_no_imputer = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_no_imputer, numeric_cols),
        ('cat', categorical_transformer_no_imputer, categorical_cols)
    ])

# Modelo
model_no_imputer = Pipeline(steps=[
    ('preprocessor', preprocessor_no_imputer),
    ('regressor', HistGradientBoostingRegressor())
    ])

# Usar cross_val_score para evaluar el modelo utilizando KFold
kfold = KFold(n_splits=10, shuffle=True, random_state=73)
results = cross_val_score(model_no_imputer, X, y, cv=kfold, scoring='r2')
print(f'R^2 promedio: {results.mean()}')

MAE promedio: 0.8970993762463035


Se comprueba que el uso de los imputadores da el mismo resultado.

De nuevo, realizaremos un RandomizedSearch para obtener los mejores **hiperparámetros**.

In [70]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', HistGradientBoostingRegressor())
    ])

# Definir el espacio de búsqueda para RandomizedSearchCV
param_dist = {
    'regressor__loss': ['squared_error', 'absolute_error', 'poisson'],
    'regressor__learning_rate': [0.01, 0.1, 0.2, 0.3],
    'regressor__max_iter': [100, 200, 300, 400, 500],
    'regressor__max_depth': [5, 10, 15, 20],
}

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    model,                                    # Pipeline
    param_distributions=param_dist,           # Hiperparámetros (espacio de búsqueda)
    verbose=1,                                # Mostrar progreso en la consola
    n_jobs=-1,                                # Usar todos los núcleos disponibles
    random_state=73                           # Reproducibilidad
)

random_search.fit(X_train, y_train)

# Mejores hiperparámetros
print("Mejores parámetros:", random_search.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Mejores parámetros: {'regressor__max_iter': 200, 'regressor__max_depth': 15, 'regressor__loss': 'poisson', 'regressor__learning_rate': 0.1}


In [71]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', HistGradientBoostingRegressor(max_depth=15, learning_rate=0.1, loss='poisson', max_iter=200))
    ])

# Ajustar el modelo
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   PowerTransformer())]),
                                                  Index(['Edad', 'IngresoBrutoAnual', 'ScoreCrediticio', 'Experiencia',
       'CuantiaCredito', 'DuracionCredito', 'NumeroPersonasDependientes',
       'PagoMensualDeuda', 'TasaUtilizacionTarjetaCredito',
       'NumeroLineasCreditoAb...
       'TipoInteres', 'PagoMensualCredito', 'RatioTotalDeudaIngresos'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['SituacionLaboral', 'NivelEducativo', 'EstadoCivil', 'EstadoVivienda',
       'ObjetoCredito'],
      dtype='object'))])),
                ('regressor',
                 HistGradientBoostingRegressor(loss='poisson', max_depth=15,
                                               max_iter=200))])

In [73]:
# Evaluar el modelo
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"R² del conjunto de entrenamiento: {train_score:.4f}")
print(f"R² del conjunto de prueba: {test_score:.4f}")

y_pred = model.predict(X_test)

# Calcular métricas para regresión
rmse = root_mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

R² del conjunto de entrenamiento: 0.9316
R² del conjunto de prueba: 0.8984
RMSE: 2.524270100136954
MAE: 1.665530297968542


## MLP Regressor